# FINAL ASSIGNMENT (for handing in)

Model modification and extension with government spending following an exogenous AR(1) process and innvation shock.

In [31]:
import numpy as np
from scipy.optimize import fsolve

sigma = 1
eta = 2
kappa = 0.3
beta = 0.995
phi_pi = 1.5
phi_y = 0.1
rho_g = 0.7
gbar = 0.25 # parameter of the AR(1) process

### Task 1
Write the model in the form A*z_t = B with z_t = [y_t, pi_t, i_t]'

As in the assignments I rewrite the model so that every variable of time t is on the left hand side.
Then I define A and B so that A*z_t = B.

## !! ad info about how g_t is handled !!

In [5]:
A = np.array([[1,0,(1-gbar)*(1/sigma)],
            [-kappa*(sigma/(1-gbar)+eta),1,0],
            [-phi_y, -phi_pi,1]])
A_inv = np.linalg.inv(A)
A

array([[ 1.  ,  0.  ,  0.75],
       [-1.  ,  1.  ,  0.  ],
       [-0.1 , -1.5 ,  1.  ]])

array([[ 1.  ,  0.  ,  0.75],
       [ -1.  ,  1.  ,  0.  ],
       [-0.1 , -1.5 ,  1.  ]])

### Task 2

Write a function, similar to the one in Assignment 2, that has as input a vector that contains ’guessed values’ of c_g and c_eta_g and gives expectations for next periods Y, pi and g.

In [30]:
# input: 'guessed values' of C_g and C_eta_g_t

def model2(coef):
    c_outcome = np.zeros(len(coef))
    B = np.zeros(3)
    c_g = coef[0:3]
    c_eta_gt = coef[3:6]
    # Calculate expectations of output, inflation and government spending via equation (6) 
    # form expectations via (6): exp_z_t = C_g * g_t + C_eta_g * 0 since exp_eta = 0 (white noise). 
    # hence we need g_t which we can take from (4). I move up the 'next, ...' -step: Set values for g_t-1 and eta_g_t:
    exp_eta_gt = 0
    eta_gt = 0.1
    g_previous = 1

    # Part 1 without shock
    g_t = rho_g * g_previous
    exp_z_t = np.dot(c_g,g_t) + np.dot(c_eta_gt,exp_eta_gt)

    expY = exp_z_t[0]
    expPi = exp_z_t[1]
    expi = exp_z_t[2]
    expG = rho_g * g_t

    # These expectation are plugged into the B vector
    B[0] = expY + (1-gbar)*1/sigma*expPi + gbar*(g_t+expG)
    B[1] = beta*expPi
    B[2] = 0
    # Calculate z_t = A_inv*B once with and without shock in B (plug (4) for g_t) to get C_g and C_eta_g_t
    c_outcome[0:3] = np.dot(A_inv,B)

    # Part 2: with shock
    g_t = rho_g * g_previous + eta_gt
    exp_z_t = np.dot(c_g,g_t) + np.dot(c_eta_gt,exp_eta_gt)

    expY = exp_z_t[0]
    expPi = exp_z_t[1]
    expi = exp_z_t[2]
    expG = rho_g * g_t

    # These expectation are plugged into the B vector
    B[0] = expY + (1-gbar)*1/sigma*expPi + gbar*(g_t+expG)
    B[1] = beta*expPi
    B[2] = 0
    # Calculate z_t = A_inv*B with shock in g_t
    c_outcome[3:6] = np.dot(A_inv,B) - c_g

    # Caculate and return difference between C_g, C_eta_g_t and the input values
    difference = c_outcome - np.array(coef)
    return difference

init = [1.1, 0.1, 0.2, 0.1, 0.2, 0.1]
model2(init)

array([-0.62652557,  0.44312443,  0.66203409, -0.65888636,  0.32071364,
        0.68518182])

### Task 3: Run fsolve to get the rational expectation solution for c_g and c_eta_gt

In [62]:
init = [1.1, 0.1, 0.2, 0.1, 0.2, 0.1]

[coefficients, inf, ier, msg] = fsolve(model2, init, full_output = True)
print(coefficients, '\n', ier,'\n', msg)
C_g = coefficients[0:3]
C_eta_gt = coefficients[3:6]

print(C_g, '\n', C_eta_gt) 
# C_g = [0.12649155 0.41677611 0.63781333] 
# C_eta_gt = [0.01807022 0.05953944 0.09111619]

[0.12649155 0.41677611 0.63781333 0.01807022 0.05953944 0.09111619] 
 1 
 The solution converged.
[0.12649155 0.41677611 0.63781333] 
 [0.01807022 0.05953944 0.09111619]


### Task 4: Calculate the response to a shock on g_t of size 0.01

Make a loop for 40 periods and set the shock to 0.01 in the first period, 0 in every other period.
In each period, calculate the variables in z_t using the vectors C_g and C_eta_gt.

In [72]:
N = 40

eta_g = np.zeros(N)
eta_g[0] = 0.01
Y = np.zeros(N)
Pi = np.zeros(N)
i = np.zeros(N)
g = np.zeros(N)

for t in range(N):
    t=t+1 # to avoid t-1 = -1 in the first period
    z = C_g*g[t-1] + C_eta_gt*eta_g[t]
    print(z)



[0.0001807  0.00059539 0.00091116]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]


In [48]:
np.dot(C_g,g[1]) + np.dot(C_eta_gt,eta_g[2])

array([0., 0., 0.])